# Restaurants recommendation 

In [205]:
import os
import json
import pandas as pd
import numpy as np
from copy import deepcopy


import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns
from pylab import savefig

from sklearn import preprocessing
from sklearn.model_selection import train_test_split
from sklearn.cluster import KMeans
from sklearn.preprocessing import LabelEncoder, MultiLabelBinarizer
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

from keras.models import Sequential, Model, load_model
from keras.utils.vis_utils import model_to_dot
from keras.optimizers import Adam
from keras.layers import Embedding, Reshape, Activation, Input, Dense, Flatten, Dropout, dot
from keras.constraints import non_neg
from IPython.display import SVG

In [206]:
Ratings_TripAdvisor = pd.read_csv('TA_restaurants_curated.csv')
Ratings_TripAdvisor.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 125527 entries, 0 to 125526
Data columns (total 11 columns):
Unnamed: 0           125527 non-null int64
Name                 125527 non-null object
City                 125527 non-null object
Cuisine Style        94176 non-null object
Ranking              115876 non-null float64
Rating               115897 non-null float64
Price Range          77672 non-null object
Number of Reviews    108183 non-null float64
Reviews              115911 non-null object
URL_TA               125527 non-null object
ID_TA                125527 non-null object
dtypes: float64(3), int64(1), object(7)
memory usage: 10.5+ MB


In [207]:
Ratings_TripAdvisor.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,URL_TA,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",/Restaurant_Review-g188590-d11752080-Reviews-M...,d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",/Restaurant_Review-g188590-d693419-Reviews-De_...,d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",/Restaurant_Review-g188590-d696959-Reviews-La_...,d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",/Restaurant_Review-g188590-d1239229-Reviews-Vi...,d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",/Restaurant_Review-g188590-d6864170-Reviews-Li...,d6864170


In [208]:
ColumnsDrop = ['URL_TA']
Ratings_v2 = Ratings_TripAdvisor.drop(ColumnsDrop, axis = 1)
Ratings_v2.head()

,Unnamed: 0,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,ID_TA
0,0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [209]:
sum(Ratings_v2['Price Range'].isnull())

47855

In [210]:
Ratings_v3 = Ratings_v2.dropna().drop(['Unnamed: 0'], axis = 1)

In [211]:
Ratings_v3.head()

,Name,City,Cuisine Style,Ranking,Rating,Price Range,Number of Reviews,Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [212]:
Ratings_v3.rename(columns=lambda x: x.replace(" ", "_"), inplace=True)
Ratings_v3.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,$$ - $$$,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,$$$$,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,$$$$,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,$$$$,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,$$$$,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [213]:
Ratings_v3['City'].unique()

array(['Amsterdam', 'Athens', 'Barcelona', 'Berlin', 'Bratislava',
       'Brussels', 'Budapest', 'Copenhagen', 'Dublin', 'Edinburgh',
       'Geneva', 'Hamburg', 'Helsinki', 'Krakow', 'Lisbon', 'Ljubljana',
       'London', 'Luxembourg', 'Lyon', 'Madrid', 'Milan', 'Munich',
       'Oporto', 'Oslo', 'Paris', 'Prague', 'Rome', 'Stockholm', 'Vienna',
       'Warsaw', 'Zurich'], dtype=object)

In [214]:
Ratings_v3['Price_Range'] = Ratings_v3['Price_Range'].apply(lambda x: len(x.split('-')[0].strip()))
Ratings_v3.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,2,136.0,"[['Just like home', 'A Warm Welcome to Wintry ...",d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,4,812.0,"[['Great food and staff', 'just perfect'], ['0...",d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,4,567.0,"[['Satisfaction', 'Delicious old school restau...",d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,4,564.0,"[['True five star dinner', 'A superb evening o...",d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,4,316.0,"[['Best meal.... EVER', 'super food experience...",d6864170


In [215]:
len(Ratings_v3), len(Ratings_v3['Name'].unique())

(74225, 67862)

In [248]:
Ratings_v4 = Ratings_v3.drop(['Reviews'], axis = 1)
Ratings_v4.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"['French', 'Dutch', 'European']",1.0,5.0,2,136.0,d11752080
1,De Silveren Spiegel,Amsterdam,"['Dutch', 'European', 'Vegetarian Friendly', '...",2.0,4.5,4,812.0,d693419
2,La Rive,Amsterdam,"['Mediterranean', 'French', 'International', '...",3.0,4.5,4,567.0,d696959
3,Vinkeles,Amsterdam,"['French', 'European', 'International', 'Conte...",4.0,5.0,4,564.0,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"['Dutch', 'European', 'International', 'Vegeta...",5.0,4.5,4,316.0,d6864170


In [249]:
Ratings_v4['Cuisine_Style'][0]

"['French', 'Dutch', 'European']"

In [250]:
import re, string
regex = re.compile('[%s]' % re.escape(string.punctuation))
def test_re(s):  # From Vinko's solution, with fix.
    return regex.sub('', s)
#Ratings_v4['Cuisine_Style'].apply(lambda x : print(x[0]) )
Ratings_v4['Cuisine_Style'] = Ratings_v4['Cuisine_Style'].apply(lambda x : [re.sub(r'[^\w\s]','',y.strip()) for y in x.strip().split(',')])
#Ratings_v4['Cuisine_Style'][0]

In [251]:
Ratings_v4.head()

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"[French, Dutch, European]",1.0,5.0,2,136.0,d11752080
1,De Silveren Spiegel,Amsterdam,"[Dutch, European, Vegetarian Friendly, Gluten ...",2.0,4.5,4,812.0,d693419
2,La Rive,Amsterdam,"[Mediterranean, French, International, Europea...",3.0,4.5,4,567.0,d696959
3,Vinkeles,Amsterdam,"[French, European, International, Contemporary...",4.0,5.0,4,564.0,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"[Dutch, European, International, Vegetarian Fr...",5.0,4.5,4,316.0,d6864170


In [252]:
grouped = Ratings_v4.groupby(['City', 'Name'])['Rating'].mean()
grouped2 = grouped.groupby(level = 'City').nlargest(50).reset_index(level= 0, drop = True)

In [253]:
#grouped.groupby(level = 'City').nlargest(50).reset_index(level= 0, drop = True)
grouped3 = grouped2.unstack(level = 0)

In [254]:
grouped2.reset_index()

,City,Name,Rating
0,Amsterdam,'t Halve Werk,5.0
1,Amsterdam,Alex + Pinard,5.0
2,Amsterdam,Amona Food Truck,5.0
3,Amsterdam,Anna's SpecialTeas,5.0
4,Amsterdam,Anne Maike of Casa Curiosa,5.0
5,Amsterdam,Arepa Shop,5.0
6,Amsterdam,Arles,5.0
7,Amsterdam,BROODJESZAAK t KUYLTJE,5.0
8,Amsterdam,Bahya Food to Share,5.0
9,Amsterdam,Baking Lab Amsterdam,5.0


In [245]:

#R = Ratings_v4.groupby(['City']).agg(sum).sort_values('Number_of_Reviews', ascending = False).head(n= 10)
#R

In [246]:
#R1 = Ratings_v4.groupby(['City']).groupby(['Name'])['Number_of_Reviews'].agg(sum).sort_values(ascending = False)
#R1
Ratings_v4.set_index(['City', 'Name'], inplace = True)

In [234]:
Ratings_v4.loc[['Amsterdam', "De Silveren Spiegel"]]

Cuisine_Style  \
City      Name                                                                                                
Amsterdam Martine of Martine's Table                                              [French, Dutch, European]   
          De Silveren Spiegel                             [Dutch, European, Vegetarian Friendly, Gluten ...   
          La Rive                                         [Mediterranean, French, International, Europea...   
          Vinkeles                                        [French, European, International, Contemporary...   
          Librije's Zusje Amsterdam                       [Dutch, European, International, Vegetarian Fr...   
          Ciel Bleu Restaurant                            [Contemporary, International, Vegetarian Frien...   
          Zaza's                                          [French, International, Mediterranean, Europea...   
          Blue Pepper Restaurant And Candlelight Cruises  [Asian, Indonesian, Vegetarian Friendly, Vegan...   
          Teppanyaki Restaurant Sazanka                   [Japanese, Asian, Vegetarian Friendly, Vegan O...   
          Rob Wigboldus Vishandel                                               [Dutch, Seafood, Fast Food]   
          The Happy Bull                                  [American, Bar, European, Vegetarian Friendly,...   
          Gartine                                         [French, Dutch, International, European, Veget...   
          Restaurant Adam                                 [French, European, Central European, Vegetaria...   
          Biercafe Gollem                                                                        [Bar, Pub]   
          Restaurant Daalder                              [French, Dutch, International, European, Veget...   
          Greenwoods Keizersgracht                        [Dutch, Cafe, European, British, Vegetarian Fr...   
          Omelegg - City Centre                           [Dutch, European, Healthy, International, Vege...   
          Brasserie Ambassade                             [French, Bar, International, European, Seafood...   
          Sherpa Restaurant                               [Indian, Tibetan, Nepali, Vegetarian Friendly,...   
          La Maschera Lillotatini                         [Italian, Mediterranean, European, Vegetarian ...   
          Senses Restaurant                                                       [International, European]   
          MOS Amsterdam                                   [French, Dutch, Seafood, European, Internation...   
          Restaurant Vlaming                              [Dutch, European, International, Vegetarian Fr...   
          Jacketz                                         [Dutch, European, Central European, Vegetarian...   
          Greenwoods Singel                               [Cafe, European, British, Vegetarian Friendly,...   
          Bord'Eau                                        [French, International, European, Vegetarian F...   
          Omelegg - De Pijp                               [Dutch, Cafe, European, Healthy, Central Europ...   
          Harmsen                                         [Dutch, European, International, Vegetarian Fr...   
          Brasserie SenT                                  [Barbecue, French, Steakhouse, European, Inter...   
          MAX Amsterdam                                   [European, Contemporary, Indonesian, Internati...   
...                                                                                                     ...   
          De Oude Taveerne Restaurant                                                     [Dutch, European]   
          Avram Grill Argentijns                                                  [Steakhouse, Argentinean]   
          Luminaa                                                                        [Bar, Pub, Fusion]   
          La Piccola Trattoria Da Tonino                                                   [Italia

In [231]:
Ratings_v4.reset_index(drop= True)

,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,"[French, Dutch, European]",1.0,5.0,2,136.0,d11752080
1,"[Dutch, European, Vegetarian Friendly, Gluten ...",2.0,4.5,4,812.0,d693419
2,"[Mediterranean, French, International, Europea...",3.0,4.5,4,567.0,d696959
3,"[French, European, International, Contemporary...",4.0,5.0,4,564.0,d1239229
4,"[Dutch, European, International, Vegetarian Fr...",5.0,4.5,4,316.0,d6864170
5,"[Contemporary, International, Vegetarian Frien...",6.0,4.5,4,745.0,d696902
6,"[French, International, Mediterranean, Europea...",7.0,4.5,2,1455.0,d1014732
7,"[Asian, Indonesian, Vegetarian Friendly, Vegan...",8.0,4.5,4,675.0,d697058
8,"[Japanese, Asian, Vegetarian Friendly, Vegan O...",9.0,4.5,4,923.0,d697009
9,"[Dutch, Seafood, Fast Food]",10.0,4.5,1,450.0,d1955652


MultiIndex(levels=[['Amsterdam', 'Athens', 'Barcelona', 'Berlin', 'Bratislava', 'Brussels', 'Budapest', 'Copenhagen', 'Dublin', 'Edinburgh', 'Geneva', 'Hamburg', 'Helsinki', 'Krakow', 'Lisbon', 'Ljubljana', 'London', 'Luxembourg', 'Lyon', 'Madrid', 'Milan', 'Munich', 'Oporto', 'Oslo', 'Paris', 'Prague', 'Rome', 'Stockholm', 'Vienna', 'Warsaw', 'Zurich'], ['"52" Bistro Restauarnt and Bar', '"Above" Roof Top Restaurant', '"Bistro Antidotum"', '"Kepzeld el!" Wine bar', '"SPECIAL" Hamburger & Italian Fast Food', '#Citypie - Fast Slow Food', '#Hashtag', '#Marionett Craft Beer House', '&Samhoud Places', ''A Pazziella', ''A livella', ''Gusto Winebar', ''Gusto-Osteria', ''O FAMO STRANO', ''O Masto', ''O Sole 'e Napule', ''Skek', ''Smiths' of Smithfield, Cannon Street', ''T Kelderke', ''cause BEER loves FOOD', ''t Blauwe Theehuis', ''t Fornuis', ''t Halve Werk', ''t Houtskooltje', ''t Loosje', ''t Nieuwe Kafe', ''t PontStation', ''t Sluisje', ''t Steakhouse', ''t Tuinfeest', ''t Verwenkaffee', 

,City,Name,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,Amsterdam,Martine of Martine's Table,"[French, Dutch, European]",1.0,5.0,2,136.0,d11752080
1,Amsterdam,De Silveren Spiegel,"[Dutch, European, Vegetarian Friendly, Gluten ...",2.0,4.5,4,812.0,d693419
2,Amsterdam,La Rive,"[Mediterranean, French, International, Europea...",3.0,4.5,4,567.0,d696959
3,Amsterdam,Vinkeles,"[French, European, International, Contemporary...",4.0,5.0,4,564.0,d1239229
4,Amsterdam,Librije's Zusje Amsterdam,"[Dutch, European, International, Vegetarian Fr...",5.0,4.5,4,316.0,d6864170
5,Amsterdam,Ciel Bleu Restaurant,"[Contemporary, International, Vegetarian Frien...",6.0,4.5,4,745.0,d696902
6,Amsterdam,Zaza's,"[French, International, Mediterranean, Europea...",7.0,4.5,2,1455.0,d1014732
7,Amsterdam,Blue Pepper Restaurant And Candlelight Cruises,"[Asian, Indonesian, Vegetarian Friendly, Vegan...",8.0,4.5,4,675.0,d697058
8,Amsterdam,Teppanyaki Restaurant Sazanka,"[Japanese, Asian, Vegetarian Friendly, Vegan O...",9.0,4.5,4,923.0,d697009
9,Amsterdam,Rob Wigboldus Vishandel,"[Dutch, Seafood, Fast Food]",10.0,4.5,1,450.0,d1955652


In [198]:
Ratings_v4

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"[French, Dutch, European]",1.0,5.0,2,136.0,d11752080
1,De Silveren Spiegel,Amsterdam,"[Dutch, European, Vegetarian Friendly, Gluten ...",2.0,4.5,4,812.0,d693419
2,La Rive,Amsterdam,"[Mediterranean, French, International, Europea...",3.0,4.5,4,567.0,d696959
3,Vinkeles,Amsterdam,"[French, European, International, Contemporary...",4.0,5.0,4,564.0,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"[Dutch, European, International, Vegetarian Fr...",5.0,4.5,4,316.0,d6864170
5,Ciel Bleu Restaurant,Amsterdam,"[Contemporary, International, Vegetarian Frien...",6.0,4.5,4,745.0,d696902
6,Zaza's,Amsterdam,"[French, International, Mediterranean, Europea...",7.0,4.5,2,1455.0,d1014732
7,Blue Pepper Restaurant And Candlelight Cruises,Amsterdam,"[Asian, Indonesian, Vegetarian Friendly, Vegan...",8.0,4.5,4,675.0,d697058
8,Teppanyaki Restaurant Sazanka,Amsterdam,"[Japanese, Asian, Vegetarian Friendly, Vegan O...",9.0,4.5,4,923.0,d697009
9,Rob Wigboldus Vishandel,Amsterdam,"[Dutch, Seafood, Fast Food]",10.0,4.5,1,450.0,d1955652


In [140]:
list(R.index)

['London',
 'Rome',
 'Paris',
 'Barcelona',
 'Milan',
 'Madrid',
 'Lisbon',
 'Prague',
 'Amsterdam',
 'Edinburgh']

In [141]:
Ratings_v5 = Ratings_v4[Ratings_v4['City'].isin(list(R.index))]
Ratings_v5

,Name,City,Cuisine_Style,Ranking,Rating,Price_Range,Number_of_Reviews,ID_TA
0,Martine of Martine's Table,Amsterdam,"[French, Dutch, European]",1.0,5.0,2,136.0,d11752080
1,De Silveren Spiegel,Amsterdam,"[Dutch, European, Vegetarian Friendly, Gluten ...",2.0,4.5,4,812.0,d693419
2,La Rive,Amsterdam,"[Mediterranean, French, International, Europea...",3.0,4.5,4,567.0,d696959
3,Vinkeles,Amsterdam,"[French, European, International, Contemporary...",4.0,5.0,4,564.0,d1239229
4,Librije's Zusje Amsterdam,Amsterdam,"[Dutch, European, International, Vegetarian Fr...",5.0,4.5,4,316.0,d6864170
5,Ciel Bleu Restaurant,Amsterdam,"[Contemporary, International, Vegetarian Frien...",6.0,4.5,4,745.0,d696902
6,Zaza's,Amsterdam,"[French, International, Mediterranean, Europea...",7.0,4.5,2,1455.0,d1014732
7,Blue Pepper Restaurant And Candlelight Cruises,Amsterdam,"[Asian, Indonesian, Vegetarian Friendly, Vegan...",8.0,4.5,4,675.0,d697058
8,Teppanyaki Restaurant Sazanka,Amsterdam,"[Japanese, Asian, Vegetarian Friendly, Vegan O...",9.0,4.5,4,923.0,d697009
9,Rob Wigboldus Vishandel,Amsterdam,"[Dutch, Seafood, Fast Food]",10.0,4.5,1,450.0,d1955652


In [124]:
#one_hot = MultiLabelBinarizer()
"""
Ratings_v5 = Ratings_v4.join(pd.DataFrame(one_hot.fit_transform(Ratings_v4.pop('Cuisine_Style')), columns = one_hot.classes_,
                                         index= Ratings_v4.index))  
Ratings_v5.head()

"""


"\nRatings_v5 = Ratings_v4.join(pd.DataFrame(one_hot.fit_transform(Ratings_v4.pop('Cuisine_Style')), columns = one_hot.classes_,\n                                         index= Ratings_v4.index))  \nRatings_v5.head()\n\n"

In [102]:
#one_hot.classes_
#Ratings_v4.join(Ratings_v5['Vegetarian Friendly'])
# Encode labels with value between 0 and n_classes-1
class MultiColumnLabelEncoder:
    def __init__(self, X, columns = None):
        self.df = deepcopy(X)
        self.columns = columns
    def fitransform(self):
        if not self.columns:            
            self.df.apply(LabelEncoder().fit_transform)
        else:
            for col in self.columns:
                #print(self.df[col])
                self.df[col] = LabelEncoder().fit_transform(self.df[col].astype(str))
        return self.df
#LE = MultiColumnLabelEncoder()

In [103]:
LE = MultiColumnLabelEncoder(Ratings_v5, ['Name', 'City', 'ID_TA'])
Ratings_v5 = LE.fitransform()

In [105]:
Ratings_v5 = Ratings_v5.iloc[:,0:7]
Ratings_v5 = Ratings_v5.rename(columns= {'Name':'Restaurants', 'ID_TA':'Users'})

In [106]:
Ratings_v5 

,Restaurants,City,Ranking,Rating,Price_Range,Number_of_Reviews,Users
0,25002,0,1.0,5.0,2,136.0,6154
1,10213,0,2.0,4.5,4,812.0,34200
2,20250,0,3.0,4.5,4,567.0,34551
3,42306,0,4.0,5.0,4,564.0,8852
4,23400,0,5.0,4.5,4,316.0,33892
5,8746,0,6.0,4.5,4,745.0,34524
6,43382,0,7.0,4.5,2,1455.0,705
7,4530,0,8.0,4.5,4,675.0,34593
8,38137,0,9.0,4.5,4,923.0,34573
9,34250,0,10.0,4.5,1,450.0,14646


In [19]:
train, test = train_test_split(Ratings_v5, test_size = 0.2)


In [20]:
# 'Restaurants', 'Users'
n_users, n_restaurant = len(Ratings_v5['Users'].unique()), len(Ratings_v5['Restaurants'].unique())
n_latent_factors = 10 # why 8 - 3 main regions, 5 different cost ranges

In [21]:
restaurant_input = Input(shape=[1],name='Item')
restaurant_embedding = Embedding(n_restaurant, n_latent_factors, name='NonNegrestaurant-Embedding', embeddings_constraint=non_neg())(restaurant_input)
restaurant_vec = Flatten(name='Flattenrestaurant')(restaurant_embedding)

user_input = Input(shape=[1],name='User')
user_vec = Flatten(name='FlattenUsers')(Embedding(n_users, n_latent_factors,name='NonNegUser-Embedding',embeddings_constraint=non_neg())(user_input))

prod = dot([restaurant_vec, user_vec], axes=1, normalize=False) #Normalized: yes or no?
model = Model([user_input, restaurant_input], prod)
model.compile('adam', 'mean_squared_error')

Instructions for updating:
Colocations handled automatically by placer.


In [22]:
model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
Item (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
User (InputLayer)               (None, 1)            0                                            
__________________________________________________________________________________________________
NonNegrestaurant-Embedding (Emb (None, 1, 10)        678620      Item[0][0]                       
__________________________________________________________________________________________________
NonNegUser-Embedding (Embedding (None, 1, 10)        741350      User[0][0]                       
__________________________________________________________________________________________________
Flattenres

In [ ]:
#add validation data in this step 
history_nonneg = model.fit([train['Users'], train['Restaurants']], 
                           train['Rating'], 
                           epochs=100, verbose=0)

In [ ]:
y_hat = np.round(model.predict([test['Users'], test['Restaurants']]),0)
y_true = test['']
mean_absolute_error(y_true, y_hat)

In [ ]:
model.save('Restaurants_cf_nonneg.h5')


In [ ]:
model = load_model('Restaurants_cf_nonneg.h5')
res = model.get_layer('NonNegrestaurant-Embedding')
res_weights = res.get_weights()[0]
lens = np.linalg.norm(res_weights, axis = 1)
normalized = (res_weights.T / lens).T

In [ ]:
def similarity_out(index):
    dists = np.dot(normalized, normalized[index])
    closest = np.argsort(dists)[::-1]
    return dists

In [ ]:
res_dict = {}
with open("dictionary_res.json", "w+") as f:
    json.dump(res_dict, f)

res_dict